In [3]:
import requests
import json
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [5]:
BALSHT = requests.get('https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=AAPL&apikey=BNM3OK9KHBBVD3YZ') #api request for balance sheet for AAPL
INCSTAT = requests.get('https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=AAPL&apikey=BNM3OK9KHBBVD3YZ') #api request for income statement for AAPL


In [7]:
bsdata = BALSHT.text
bsparsed = json.loads(bsdata)
bsquartdata = bsparsed['quarterlyReports']
bsquartdf = pd.DataFrame.from_dict(bsquartdata)

isdata = INCSTAT.text
isparsed = json.loads(isdata)
isquartdata = isparsed['quarterlyReports']
isquartdf = pd.DataFrame.from_dict(isquartdata)

In [9]:
response = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=aapl&outputsize=full&apikey=BNM3OK9KHBBVD3YZ')
data = response.text
parsed = json.loads(data)
tsdata = parsed['Time Series (Daily)']
tsda = pd.DataFrame.from_dict(tsdata, orient='index')
tsda_new=tsda.loc[:'2009-01-02',]

In [11]:
bsolddf = pd.read_excel("AAPL BAL SHEET.xls")

isolddf = pd.read_excel("AAPL INCOME SHEET.xls")

In [13]:
assets_liab_new = bsquartdf[['fiscalDateEnding', 'totalAssets', 'totalLiabilities']]
assets_liab_new['fiscalDateEnding'] = pd.to_datetime(assets_liab_new['fiscalDateEnding'])
assets_liab_new[['totalAssets', 'totalLiabilities']] = assets_liab_new[['totalAssets', 'totalLiabilities']].apply(pd.to_numeric)
assets_liab_old = bsolddf[['Period End Date', 'TOTAL ASSETS', 'TOTAL LIABILITIES']]
new_cols = {x: y for x, y in zip(assets_liab_old.columns, assets_liab_new.columns)}
assets_liab1 = assets_liab_new.append(assets_liab_old.rename(columns=new_cols))
assets_liab = assets_liab1.set_index('fiscalDateEnding')

assets_liab.loc['2019-06-30	', 'totalLiabilities'] = 225783000000
assets_liab.loc['2019-03-31', 'totalLiabilities'] = 236138000000
assets_liab.loc['2018-12-31', 'totalLiabilities'] = 255827000000
assets_liab.loc['2018-09-30', 'totalLiabilities'] = 258578000000

<ipython-input-13-f16ba6565ca0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assets_liab_new['fiscalDateEnding'] = pd.to_datetime(assets_liab_new['fiscalDateEnding'])


In [32]:
#sb.lineplot(data=assets_liab, x="fiscalDateEnding", y="totalAssets")

In [33]:
profits_new = isquartdf[['fiscalDateEnding', 'grossProfit']]
profits_new['fiscalDateEnding'] = pd.to_datetime(profits_new['fiscalDateEnding'])
profits_new['grossProfit'] = pd.to_numeric(profits_new['grossProfit'])
profits_old = isolddf[['Period End Date', 'Gross Profit']]
new_cols = {x: y for x, y in zip(profits_old.columns, profits_new.columns)}
profits1 = profits_new.append(profits_old.rename(columns=new_cols))
profits = profits1.set_index('fiscalDateEnding')



<ipython-input-33-034380a045d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profits_new['fiscalDateEnding'] = pd.to_datetime(profits_new['fiscalDateEnding'])
<ipython-input-33-034380a045d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profits_new['grossProfit'] = pd.to_numeric(profits_new['grossProfit'])


In [35]:
assets_liab_ordinal = assets_liab.copy(deep=True)
assets_liab_ordinal.index = assets_liab_ordinal.index.map(dt.datetime.toordinal)
profits_ordinal = profits.copy(deep=True)
profits_ordinal.index = profits_ordinal.index.map(dt.datetime.toordinal)

db_clean = pd.DataFrame({'Date': tsda_new.index, 'totalAssets': np.nan, 'totalLiabilities': np.nan, 'grossProfit': np.nan})
db_clean['Date'] = pd.to_datetime(db_clean['Date'])
db_clean = db_clean.set_index('Date')
db_clean.index = db_clean.index.map(dt.datetime.toordinal)

In [39]:
i = 0
date = assets_liab_ordinal.index[i]

for d in db_clean.index:
    if d >= date:
        db_clean.loc[d, 'totalAssets'] = assets_liab_ordinal.iloc[i, 0]
        db_clean.loc[d, 'totalLiabilities'] = assets_liab_ordinal.iloc[i, 1]
        db_clean.loc[d, 'grossProfit'] = profits_ordinal.iloc[i, 0]
    else:
        i = i+1
        date = assets_liab_ordinal.index[i]
        db_clean.loc[d, 'totalAssets'] = assets_liab_ordinal.iloc[i, 0]
        db_clean.loc[d, 'totalLiabilities'] = assets_liab_ordinal.iloc[i, 1]
        db_clean.loc[d, 'grossProfit'] = profits_ordinal.iloc[i, 0]
